`conda install -c anaconda yaml`
`conda install -c conda-forge pydub`

In [30]:
import yaml
from pydub import AudioSegment  # You can open and save WAV files with pure python
from random import random
import os

test_fraction = 0.2
duration = 1000 # in milliseconds
overlap = 100

def build_directory_structure(annotation_dict):
    types = {}
    for annotation_list in annotation_dict.values():
        for entry in annotation_list:
            types[entry["type"]] = 1
    for type in types:
        test_dir = f"data/test/{type}"
        train_dir = f"data/train/{type}"
        if not os.path.exists(test_dir):
            os.makedirs(test_dir)
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)


def split_to_file(segment, name, start_file_number):
    splitFileCount = 1
    start = 0
    fullDuration = segment.duration_seconds * 1000

    while (start + duration) <= fullDuration:
        end = start + duration
        audioSlice = segment[start:end]
        audioSlice = audioSlice.set_channels(1) #convert to mono
        audioSlice = audioSlice.set_frame_rate(16000) # 16 kHz
        trainOrTest = "train" if random() > test_fraction else "test"
        resultFile = f"./data/{trainOrTest}/{name}/{start_file_number + splitFileCount}.wav"
        audioSlice.export(out_f=resultFile, format="wav")
        splitFileCount += 1
        start = start + duration - overlap
    return splitFileCount


annotation_file = "data/annotations.yaml"
with open(annotation_file, 'r') as file:
    annotations = yaml.safe_load(file)
build_directory_structure(annotations)

generated_file_count = 0
for audio_file in annotations.keys():
    print(f"Processing {annotations[audio_file]}")
    audio_input = AudioSegment.from_wav(f"data/raw/{audio_file}")
    for annotation in annotations[audio_file]:
        start = annotation["start"] * 1000 + 1 # start position is not included, see yaml annotation
        end = annotation["end"] * 1000
        current_segment = audio_input[start:end]
        generated_file_count += split_to_file(current_segment, annotation["type"], generated_file_count)
    print(f"\tFile ids to {generated_file_count}")

Processing [{'start': 0, 'end': 5, 'type': 'Background'}, {'start': 5, 'end': 7, 'type': 'Sniffing'}]
Generated 9 total files
